In [1]:
import torch
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration,Blip2Processor, Blip2ForConditionalGeneration
from glob import glob
from tqdm import tqdm
import pandas as pd

In [2]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value
        
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16).to("cuda")

/root/miniconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
images = sorted(glob('./images/val/*.jpg'))
captions = []
i=0
for image in tqdm(images):
    if i % 100 == 0:
        df = pd.DataFrame({'image': images[:i], 'caption': captions})
        df.to_csv(f'../captions_{i}.csv', index=False)
    i+=1
    raw_image = Image.open(image).convert('RGB')
    image = processor(raw_image, return_tensors="pt").to("cuda", torch.float16)
    output = model.generate(**image)
    caption = processor.decode(output[0], skip_special_tokens=True)
    captions.append(caption)
    
df = pd.DataFrame({'image': images, 'caption': captions})
df.to_csv('../captions.csv', index=False)

100%|██████████| 369/369 [02:26<00:00,  2.51it/s]
